# Keener and Sneyd Blood Model


In [19]:
import numpy as np
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.layouts import row, widgetbox, gridplot
from bokeh.models import CustomJS, Slider, Range1d, LabelSet, Label
import warnings

warnings.simplefilter('ignore')


def saturationFxn(oPP, kp, expTerm):
    percentSaturation = ( (oPP**expTerm) / ((kp**expTerm)+(oPP**expTerm)) ) * 100
    return percentSaturation

oPP = np.arange(0,120,1)

kp_mb = 2.6
n_mb = 1
myoglobinPercentSaturation  = saturationFxn(oPP, kp_mb, n_mb)

kp_hb = 26
n_hb = 2.5
hemoglobinPercentSaturation = saturationFxn(oPP, kp_hb, n_hb)

kp_gen = 2.4
expTerm_gen = 1
genericPercentSaturation = saturationFxn(oPP, kp_gen, expTerm_gen)

source = ColumnDataSource(data=dict(oPP=oPP, myoglobinPercentSaturation=myoglobinPercentSaturation, 
                                    hemoglobinPercentSaturation=hemoglobinPercentSaturation))


genericCasePlot = figure(title = "Keener & Sneyd Blood Model")

# make invisible points at minima and maxima to fix the plot
genericCasePlot.scatter(np.max(oPP),100,alpha=0)
genericCasePlot.scatter(0,0,alpha=0)
genericCasePlot.xaxis.axis_label = "Oxygen Partial Pressure (mm Hg)"
genericCasePlot.yaxis.axis_label = "Saturation (%)"

genericCasePlot.xaxis.major_label_text_font_style = 'bold'
genericCasePlot.xaxis.axis_label_text_font_size = '20pt'

genericCasePlot.yaxis.major_label_text_font_style = 'bold'
genericCasePlot.yaxis.axis_label_text_font_size = '20pt'

genericCasePlot.line('oPP','myoglobinPercentSaturation', source = source, line_width = 3,
                     line_alpha = 1, line_color = "red" , legend_label = 'Mb')
genericCasePlot.line('oPP','hemoglobinPercentSaturation', source = source, line_width = 3,
                     line_alpha = 1, line_color = "blue", legend_label = 'Hb')

kp_mb_slider = Slider(start=0, end= 50, value= 2.6, step= 0.1, title="K_Mb")
kp_hb_slider = Slider(start=0, end= 50, value= 26, step= 0.1, title="K_Hb")

n_mb_slider = Slider(start=0, end= 50, value= 1, step= 0.1,title="n_Mb")
n_hb_slider = Slider(start=0, end= 50, value= 2.5, step= 0.1,title="n_Hb")

callback = CustomJS(args=dict(source=source, kp_mb_ = kp_mb_slider, kp_hb_ = kp_hb_slider, 
                              n_mb_ = n_mb_slider, n_hb_ = n_hb_slider), code="""
    var data = source.data;
    var oPP = data['oPP']
    var myoglobinPercentSaturation = data['myoglobinPercentSaturation']
    var hemoglobinPercentSaturation = data['hemoglobinPercentSaturation']
    var kp_mb = kp_mb_.value
    var n_mb = n_mb_.value
    var kp_hb = kp_hb_.value
    var n_hb = n_hb_.value    
    for (var indexVal = 0; indexVal < oPP.length; indexVal++) {
        myoglobinPercentSaturation[indexVal] = (( (oPP[indexVal]** n_mb) / ((kp_mb** n_mb)+(oPP[indexVal]** n_mb)) ) ) * 100;
        hemoglobinPercentSaturation[indexVal] = (( (oPP[indexVal]** n_hb) / ((kp_hb** n_hb)+(oPP[indexVal])** n_hb) ) ) * 100;
    }
    source.change.emit();
""")

kp_mb_slider.js_on_change('value', callback)
kp_hb_slider.js_on_change('value', callback)

n_mb_slider.js_on_change('value', callback)
n_hb_slider.js_on_change('value', callback)

genericCasePlot.legend.location= "bottom_right"
genericCasePlot.legend.title_text_font_style = "bold"
genericCasePlot.legend.title_text_font_size = "15pt"

layout = row(
    genericCasePlot,
    widgetbox(kp_mb_slider,n_mb_slider, kp_hb_slider,n_hb_slider)
)

output_notebook()


show(layout)

Loading BokehJS ...